# Ray Concepts - Task Parallelism (Part 2)

The previous lesson explored Ray's core concepts and how they work. We learned how to define Ray _tasks_, run them, and retrieve the results. We also started learning about how Ray schedules tasks in a distributed environment.

This lesson completes the discussion of Ray tasks by exploring how task dependencies are handled. We'll finish with a look under the hood at Ray's architecture and runtime behavior.

> **Tip:** Recall that the [Ray Package Reference](https://ray.readthedocs.io/en/latest/package-ref.html) in the [Ray Docs](https://ray.readthedocs.io/en/latest/) is useful for exploring the API features we'll learn.

In [1]:
# If you are running on Google Colab, uncomment and run the following linkes
# to install the necessary dependencies.

# print("Setting up colab environment")
# !pip install -q ray
# !pip install -q bokeh

In [2]:
# Imports and initialize Ray. We're adding NumPy for the examples and the tutorial `util` library:

import ray, time, sys    # New notebook, so new process
import numpy as np       # Used for examples

In [3]:
def pnd(n, duration, prefix=''):
    """Print an integer and a time duration, with an optional prefix."""
    prefix2 = prefix if len(prefix) == 0 else prefix+' '
    print('{:s}n: {:2d}, duration: {:6.3f} seconds'.format(prefix2, n, duration))

def pd(duration, prefix=''):
    """Print a time duration, with an optional prefix."""
    prefix2 = prefix if len(prefix) == 0 else prefix+' '
    print('{:s}duration: {:6.3f} seconds'.format(prefix2, duration))

In [4]:
ray.init(ignore_reinit_error=True)

2020-04-14 12:23:18,743	INFO resource_spec.py:212 -- Starting Ray with 12.89 GiB memory available for workers and up to 6.47 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-14 12:23:19,074	INFO services.py:1148 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.102',
 'redis_address': '192.168.1.102:52572',
 'object_store_address': '/tmp/ray/session_2020-04-14_12-23-18_734034_51466/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-14_12-23-18_734034_51466/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-04-14_12-23-18_734034_51466'}

Let's work with a new remote function. Previously, our `expensive` and `expensive_task` functions returned tuples that included time durations. Obviously the durations were useful for understanding how long the functions took to execute. Now, it will be more convenient to not return "metadata" like this, but just data values that we care about, because we are going to pass them to other functions. 

Hence, we'll define _dependency_ relationships between tasks. We'll learn how Ray handles these dependent, asynchronous computations.

So, let's define a task to return a random NumPy array of some size `n`. As before, we'll add a sleep time, one tenth the size of `n`:

In [5]:
@ray.remote
def make_array(n):
    time.sleep(n/10.0)
    return np.random.standard_normal(n)

Now define a task that can add two NumPy arrays together. The arrays need to be the same size, but we'll ignore any checking for this requirement.

In [6]:
@ray.remote
def add_arrays(a1, a2):
    time.sleep(a1.size/10.0)
    return np.add(a1, a2)

Now lets use them!

In [7]:
start = time.time()
id1 = make_array.remote(20)
id2 = make_array.remote(20)
id3 = add_arrays.remote(id1, id2)
print(ray.get(id3))
pd(time.time() - start, prefix="Total time:")

[ 0.84089999  0.2894145  -0.41857772 -1.7385907  -0.15316508 -0.57480122
 -0.07055859  0.62629295  0.23034914 -0.46086276  0.13723095  0.32651502
  0.0738438  -1.60176748  0.82177929  2.27422636 -0.4728901  -0.20257597
  1.96032038 -0.42992837]
Total time: duration:  4.446 seconds


Something subtle and "magical" happened here; when we called `add_arrays`, we didn't need to call `ray.get()` first for `id1` and `id2`, since `add_arrays` expects NumPy arrays. Because `add_arrays` is a Ray task, Ray automatically does the extraction for us, so we can write code that looks more natural.

Furthermore, note that the `add_arrays` task effectively depends on the outputs of the two `make_array` tasks. Ray won't run `add_arrays` until the other tasks are finished. Hence, _Ray handles task dependencies automatically for us._ 

This is why the elapsed time is about 4 seconds. We used a size of 20, so we slept 2 seconds in each call to `make_array`, but those happened in parallel, _followed_ by a second sleep of 2 seconds in `add_arrays`.

Recall from the previous lesson that we explored when to call `ray.get()` to avoid forcing tasks to become synchronous when they should be asynchronous. This additional example illustrates two key points:

* _Don't ask for results you don't need._
* _Don't ask for the results you need until you really need them._

We don't need to see the objects for `id1` and `id2`. We only need the final array for `id3`.

## Using ray.wait() with ray.get()

We've seen several examples of the best idiomatic way to use `ray.get()`. Here again is an example from the last lesson:

```python
start = time.time()
ids = [expensive_task.remote(n) for n in range(5)]  # Fire off the asynchronous tasks
for n2, duration in ray.get(ids):                   # Retrieve all the values from the list of futures
    p(n2, duration)
pd(time.time() - start, prefix="Total time:")
```

Let's try it again with our new methods:

In [8]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]
for array in ray.get(added_array_ids):
    print(f'{array.size}: {array}')
pd(time.time() - start, prefix="Total time:")

0: []
10: [-1.92797832  1.95260875  0.50021032 -0.4231631   1.74587166 -0.1861977
 -2.96400132  2.17452547  2.72229121  2.11126325]
20: [-3.63185912  0.57926293 -3.36886248  2.1888112   0.68773383  1.03092107
  0.10188319 -3.43028407 -0.88879433 -2.07343685  3.19196529  2.30051676
  0.05581076 -1.5582634  -1.83461842 -1.2991064  -2.40962068  2.96261424
  3.00909633 -0.03967563]
30: [ 0.12791515  1.5930508  -0.93510234  3.80467391  1.04781008  1.28471091
 -1.79131811  0.70796773 -2.92337859  1.28743656 -1.71728633 -0.55546914
 -1.55260746 -0.66514647  0.17977526  1.45748877 -0.30902416 -3.18517745
  1.71314377 -0.13121606  1.71207025 -0.45387761 -0.3392279  -1.65964356
  0.9085188  -4.3109494   1.90223857  1.67443224  2.16355263  2.56295073]
40: [ 2.79758217 -3.65734333  3.12404525  2.16586443  0.69853902  1.24070785
  1.28525314  0.94317539  3.8085678  -2.1863611   0.16209508  1.7271124
 -0.71902315  0.73240667  1.07188498 -0.20065888 -1.11949972  0.85116783
  2.48393083 -0.21021716 -1

On my machine, I waited 8 seconds and then everything was printed at once.

There are two fundamental problems with the way we've used `ray.get()` so far:

1. There's no timeout, in case something gets "hung".
2. We have to wait for _all_ the objects to be available before `ray.get()` returns.

The ability to specify a timeout is essential in production code as a defensive measure. Many potential problems could happen in a real production system, any one of which could cause the task we're waiting on to take an abnormally long time to complete or never complete. Our application would be deadlocked waiting on this task. Hence, it's **strongly recommended** in production software to always use timeouts on blocking calls, so that the application can attempt some sort of recovery in situations like this, or at least report the error and "degrade gracefully".

Actually, there _is_ a `timeout=<value>` option you can pass to `ray.get()` ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.get)), but it will most likely be removed in a future release of Ray. Why remove it if timeouts are important? This change will simplify the implementation of `ray.get()` and encourage the use of `ray.wait()` for waiting ([documentation](https://ray.readthedocs.io/en/latest/package-ref.html#ray.wait)) instead, followed by using `ray.get()` to retrieve values for tasks that `ray.wait()` tells us are finished. 

Using `ray.wait()` is also the way to fix the second problem with using `ray.get()` by itself, that we have to wait for all tasks to finish before we get any values back. Some of those tasks might finish quickly, like our contrived examples that sleep for short durations compared to other invocations. 

When you have a list of asynchronous tasks, you want to process the results of them as soon they become available, even while others continue to run. Use `ray.wait()` for this purpose.

Therefore, while `ray.get()` is simple and convenient, for _production code_, we recommend using `ray.wait()`, **with** timeouts, for blocking on running tasks. Then use `ray.get()` to retrieve values of completed tasks. Now we'll learn how to use these two together. For a longer discussion on `ray.wait()`, see [this blog post](https://medium.com/distributed-computing-with-ray/ray-tips-and-tricks-part-i-ray-wait-9ed7a0b9836d).

Here is the previous example rewritten to use `ray.wait()`:

In [9]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as one of them completes,
    #   3. tell it wait up to 10 seconds before timing out.
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=1, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   1 completed tasks. (elapsed time:  0.004)
0: []
Returned   1 completed tasks. (elapsed time:  2.011)
10: [-0.85449298  1.15447431 -2.60940523 -1.75740149 -2.41089874  1.47905051
 -0.23022805  0.21967976  1.22964948 -2.04283504]
Returned   1 completed tasks. (elapsed time:  4.009)
20: [-1.21487318  2.56244402 -1.23654332 -3.28333006 -1.20740969  2.39720705
  2.9384196  -3.47694866 -0.67281025 -2.16697581  0.4629965   0.09429516
 -2.47184897 -0.16357379  2.72500368 -0.99202754 -2.67189239 -4.45407665
  1.23842453  0.68647595]
Returned   1 completed tasks. (elapsed time:  6.009)
30: [-3.11093129  1.0825721   2.30213175  2.31177037 -2.65396368  1.88527072
  1.33445319  2.61171615 -1.13107125  2.68175946  2.24242472  2.15692703
 -0.77923656  3.28037828 -2.45808294  1.48073312  0.13788289  4.3452465
  0.16205333  0.38917936  1.08043093  1.72138918 -1.42621375 -3.44727482
  3.52339114  0.48044016 -1.02795655 -0.57662428  0.85252907  2.31112521]
Returned   1 completed tasks. (elapse

Now it still takes about 8 seconds to complete, 4 seconds for the longest invocation of `make_array` and 4 seconds for the invocation of `add_arrays`, but since the others complete more quickly, we see their results as soon as they become available, at 0, 2, 4, and 6 second intervals.

> **Warning:** For each call to `ray.wait()` in a loop like this, it's important to remove the ids that have completed. Otherwise, `ray.wait()` will return immediately with the same list containg the first completed item, over and over again; you'll loop forever!! Resetting the list is easy, since the second list returned by `ray.wait()` is the rest of the items that are still running. So, that's what we use.

Now let's try it with `num_returns = 2`:

In [10]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=10.0)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.006)
0: []
10: [-1.86926937 -2.79755426 -2.88371951  0.51329029  2.04036461  2.51562627
  2.76583854 -1.36741451  2.43365363  0.55927884]
Returned   2 completed tasks. (elapsed time:  6.014)
20: [-0.54269521 -1.84024491 -0.34883852  2.91032434  1.30178174  0.47346447
 -1.25463393  1.32749867  1.0066404  -3.47148367 -0.71084499  0.53963217
 -0.5210317  -2.20719988  0.18085195 -1.05244039  1.81690414  4.52141655
  0.57138735  3.45802739]
30: [-1.25480595 -1.81101011  3.47889852 -1.87615977 -1.80450219 -0.75241735
 -2.00089927 -0.41602412 -0.99066026  0.42769706 -1.4573502  -1.67251474
  1.18773697 -0.3951468  -2.28158217 -0.48963962 -2.39306307  0.42585973
 -0.28605042 -2.31749175  1.13693264  2.13648719  1.35717073  1.35160299
  0.34976972  1.99357586  2.08799347  0.05368404 -0.0529422   0.65635089]
Returned   1 completed tasks. (elapsed time:  8.011)
40: [-3.34326143 -3.74834345  1.64783881 -2.29707798 -0.94796681  2.06371446
 -1.06037683

Now we get two at a time output. Note that we don't actually pass `num_returns=2` every time. If you ask for more items than the length of the input list, you get an error. So, we compute `num_returns`, using `2` except when there's only one task to wait on, in which case we use `1`. So, in fact, the output for `40` was a single task result, because we started with `5` and processed two at a time.

## Exercise 2

The following cell is identical to the last one. Modify it to use a timeout of `2.5` seconds, shorter than our longest tasks. What happens now? Try using other times.

In [11]:
start = time.time()
array_ids = [make_array.remote(n*10) for n in range(5)]
added_array_ids = [add_arrays.remote(id, id) for id in array_ids]

arrays = []
waiting_ids = list(added_array_ids)        # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=2.5)
    print('Returned {:3d} completed tasks. (elapsed time: {:6.3f})'.format(len(ready_ids), time.time() - start))
    new_arrays = ray.get(ready_ids)
    arrays.extend(new_arrays)
    for array in new_arrays:
        print(f'{array.size}: {array}')
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!
    
print(f"\nall arrays: {arrays}")
pd(time.time() - start, prefix="Total time:")

Returned   2 completed tasks. (elapsed time:  2.010)
0: []
10: [-0.18709138 -1.86872945  1.91816065  0.7808244  -2.72476331  1.14352354
  0.81234357  0.68776441 -0.64189843  1.08302092]
Returned   1 completed tasks. (elapsed time:  5.525)
20: [ 0.52266716 -3.08460851  0.59384596  1.89934035 -0.99747204  0.87197793
  2.02192592 -0.06347615  1.25861112  2.39331936  0.4994623   0.96452075
  1.75120455  1.53829654 -2.13063102  0.79245919 -0.93726517  1.61269672
  1.87419005 -0.25162768]
Returned   2 completed tasks. (elapsed time:  8.008)
30: [ 1.16540135  2.1067562   0.52886171 -2.11710667 -3.99711496 -1.62270596
  1.31864812  2.26691929 -2.53565935 -1.67409131  0.63335523 -0.08132226
  2.62662533 -0.95682833 -1.40950228  0.19723     1.49864354 -2.10594921
 -0.94308719 -2.73808645  2.3044671  -4.4851769   3.21948042  1.66468726
 -2.5320676   0.89294762  2.10017591 -0.89444679 -1.43175178  3.12545499]
40: [-2.60447255 -4.90947643  0.6257831  -0.64722309  1.9121866  -1.20574243
  3.2662678 

In conclusion:

> **Tips:**
>
> 1. Use `ray.wait()` with a timeout to wait for one or more running tasks. Then use `ray.get()` to retrieve the values for the finished tasks.
> 2. Don't ask for results you don't need.
> 3. Don't ask for the results you need until you really need them.

## Exercise 3

Let's make sure you understand how to use `ray.wait()`. The definitions from Exercise 1 in the previous lesson are repeated in the next cell. Change the definitions to use Ray. In particular, use `ray.wait()` as we used it above. You can just use the default values for `num_returns` and `timeout` if you want. The second cell uses `assert` statements to check your work.

> **Tip:** The solution is in the `solutions` folder.

In [12]:
def slow_square(n):
    time.sleep(n)
    return n*n

@ray.remote
def fast_square(n):
    return slow_square(n)

start = time.time()
square_ids = [fast_square.remote(n) for n in range(4)]
squares = []

waiting_ids = list(square_ids)             # Assign a working list to the full list of ids
while len(waiting_ids) > 0:                # Loop until all tasks have completed
    # Call ray.wait with:
    #   1. the list of ids we're still waiting to complete,
    #   2. tell it to return immediately as soon as TWO of them complete,
    #   3. tell it wait up to 10 seconds before timing out.
    return_n = 2 if len(waiting_ids) > 1 else 1
    ready_ids, remaining_ids = ray.wait(waiting_ids, num_returns=return_n, timeout=2.5)
    new_squares = ray.get(ready_ids)
    squares.extend(new_squares)
    waiting_ids = remaining_ids  # Reset this list; don't include the completed ids in the list again!

duration = time.time() - start

In [13]:
assert squares == [0, 1, 4, 9], f'Did you use ray.get() to retrieve the values? squares = {squares}'
assert duration < 4.1, f'Did you use Ray to parallelize the work? duration = {duration}' 

## What Is the Optimal Task Granularity

How fine-grained should Ray tasks be? There's no fixed rule of thumb, but Ray clearly adds some overhead for task management and using object stores in a cluster. Therefore, it makes sense that tasks which are too small will perform poorly.

We'll explore this topic over several more lessons, but for now, let's get a sense of the overhead while running in your setup.

We'll continue to use NumPy arrays to create "load", but remove the `sleep` calls:

In [14]:
def noop(n):
    return n

def local_make_array(n):
    return np.random.standard_normal(n)

@ray.remote
def remote_make_array(n):
    return local_make_array(n)

Let's do `trials` runs for each experiment, to average out background noise:

In [15]:
trials=100

First, let's use `noop` to baseline local function calls. Note that we call `print` for the duration, rathern than `pd`, because the overhead is so low the `pd` formatting will print `0.000`:

In [16]:
start = time.time()
[noop(t) for t in range(trials)]
print(f'{time.time() - start} seconds')

0.00010609626770019531 seconds


Let's try the same run with `local_make_array(n)` for `n = 100000`:

In [17]:
start = time.time()
[local_make_array(100000) for _ in range(trials)]
print(f'{time.time() - start} seconds')

0.2957029342651367 seconds


So, we can safely ignore the "noop" overhead for now. For completeness, here's what happens with remote execution:

In [18]:
start = time.time()
ids = [remote_make_array.remote(100000) for _ in range(trials)]
ray.get(ids)
print(f'{time.time() - start} seconds')

0.08694195747375488 seconds


For arrays of 100000, using Ray is faster (at least on this test machine). The benefits of parallel computation, rather than synchronous, already outweight the Ray overhead.

So, let's run some trials with increasingly large array sizes, to compare the performance with local vs. remote execution. First, we'll set up `matplotlib`:

In [19]:
local_durations = []
remote_durations = []
# These n values were determined by experimentation on this test machine. 
# If you are using an old machine, and this cell takes a long time to execute,
# you could set the `trials` value above to a smaller number. 
ns = [i*(10**j) for j in range(2,5) for i in [1,2,3,5,8]]
for n in ns:
    start_local = time.time()
    [local_make_array(n) for _ in range(trials)]
    local_durations.append(time.time() - start_local)
    
    start_remote = time.time()
    ids = [remote_make_array.remote(n) for _ in range(trials)]
    ray.get(ids)
    remote_durations.append(time.time() - start_remote)
(ns, local_durations, remote_durations)

([100,
  200,
  300,
  500,
  800,
  1000,
  2000,
  3000,
  5000,
  8000,
  10000,
  20000,
  30000,
  50000,
  80000],
 [0.0014541149139404297,
  0.0010333061218261719,
  0.0012080669403076172,
  0.0019462108612060547,
  0.0029458999633789062,
  0.003633260726928711,
  0.006973981857299805,
  0.008816242218017578,
  0.015500068664550781,
  0.02538013458251953,
  0.03076934814453125,
  0.06094002723693848,
  0.08382797241210938,
  0.13878917694091797,
  0.2144918441772461],
 [0.027678966522216797,
  0.022787809371948242,
  0.0201570987701416,
  0.019936084747314453,
  0.019878864288330078,
  0.020009994506835938,
  0.019706010818481445,
  0.020267724990844727,
  0.02091693878173828,
  0.021129131317138672,
  0.02340388298034668,
  0.02861618995666504,
  0.034046173095703125,
  0.0418851375579834,
  0.0546109676361084])

In [20]:
import numpy as np

from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show

import bokeh.io
# The next two lines prevent Bokeh from opening the graph in a new window.
bokeh.io.reset_output()
bokeh.io.output_notebook()

tooltips = [
    ("name", "$name"),
    ("array size", "$x"),
    ("time", "$y")]
p1 = figure(x_axis_type="log", y_axis_type="log", title="Execution Times", tooltips=tooltips)
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'array size'
p1.yaxis.axis_label = 'time'

p1.line(ns, local_durations, color='#A6CEE3', legend_label='local', name='local')
p1.circle(ns, local_durations, color='darkgrey', size=4)
p1.line(ns, remote_durations, color='#B2DF8A', legend_label='remote', name='remote')
p1.square(ns, remote_durations, color='darkgrey', size=4)
p1.legend.location = "top_left"

show(gridplot([[p1]], plot_width=800, plot_height=400))

Loading BokehJS ...

Here's a static image from a test run, in case the Bokeh plot isn't working. Your results may look a lot different!
![Execution Times: Local versus Remote](../images/Execution-Times-Local-v-Remote.png)

Let's confirm what the graph shows as the crossing point:

In [21]:
i=0
while i < len(ns) and local_durations[i] < remote_durations[i]:
    i=i+1
print('The Ray times are faster starting at n = {:d}, local = {:6.3f} vs. remote = {:6.3f}'.format(
    ns[i], local_durations[i], remote_durations[i]))

The Ray times are faster starting at n = 8000, local =  0.025 vs. remote =  0.021


## How Distributed Task Management Works

> **Note:** If you just want to learn the Ray API, you can safely skip the rest of this lesson (notebook) for now. It continues the exploration of how Ray works internally, which we started in the previous lesson. However, you should come back to this material at some point, so you'll develop a better understanding of how Ray works.

At the end of the last lesson, we examined Ray task scheduling at a high-level, by watching the Ray Dashboard and analyzing the performance times. Now we'll walk through some images that show the process Ray follows to place tasks around a cluster. 

Assume we will invoke the `make_array` task twice, then invoke `add_arrays` to sum the returned NumPy arrays. Graphically, it looks as follows:

![Ray under the hood 1](../images/Ray-Cluster/Ray-Cluster.001.jpeg)

How does this get scheduled in a cluster? Here we'll assume a three-node cluster that has resources for running two Ray worker tasks per node (under powered compared to what we learned using Ray Dashboard last lesson!).
![Ray under the hood 2](../images/Ray-Cluster/Ray-Cluster.002.jpeg)

First, assume that the driver program is running on Node1. So it will invoke the local scheduler to schedule the three tasks.
![Ray under the hood 3](../images/Ray-Cluster/Ray-Cluster.003.jpeg)

Immediately the ids for the task futures are returned. The _Global Control Store_ tracks where every task is running and every object is stored in the local _Object Stores_.
![Ray under the hood 4](../images/Ray-Cluster/Ray-Cluster.004.jpeg)

Suppose the local scheduler has available capacity in the first worker on the same node. It schedules the first `make_array` task there.
![Ray under the hood 5](../images/Ray-Cluster/Ray-Cluster.005.jpeg)

It decides to schedule the second `make_array` task in a worker on node 2.
![Ray under the hood 6](../images/Ray-Cluster/Ray-Cluster.006.jpeg)

When the two tasks finish, they place their result objects in their local object stores.
![Ray under the hood 7](../images/Ray-Cluster/Ray-Cluster.007.jpeg)

Now `add_array` can be scheduled, because the two tasks it depends on are done. Let's suppose it gets scheduled in the second worker on Node 1.
![Ray under the hood 8](../images/Ray-Cluster/Ray-Cluster.008.jpeg)

The first object it needs is already on the same node, in the object store, so the `add_arrays` task can _read it directly from shared memory_. No copying is required to the worker's process space.
![Ray under the hood 9](../images/Ray-Cluster/Ray-Cluster.009.jpeg)

However, the second object is on a different node, so Ray copies it to the local object store. 
![Ray under the hood 10](../images/Ray-Cluster/Ray-Cluster.010.jpeg)

Now it can also be read from shared memory.
![Ray under the hood 11](../images/Ray-Cluster/Ray-Cluster.011.jpeg)

When `add_arrays` is finished, it writes its results to the local object store.
![Ray under the hood 12](../images/Ray-Cluster/Ray-Cluster.012.jpeg)

At this point, if the driver calls `ray.get(id3)`, it will return `obj3`.
![Ray under the hood 13](../images/Ray-Cluster/Ray-Cluster.013.jpeg)

Whew! Hopefully you have a better sense of what Ray does under the hood. Scheduling tasks on other nodes and copying objects between object stores is efficient, but incurs unavoidable network overhead.